In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import IntegerType,StructField,StructType, BooleanType
from pyspark.sql.functions import col,sum,udf,when

spark = SparkSession.builder.appName("app").master("local[*]").getOrCreate()

In [0]:
schema = StructType([
    StructField("x",IntegerType(),True),
    StructField("y",IntegerType(),True),
    StructField("z",IntegerType(),True)
])

data = [
    (13,15,30),
    (10,20,15)
]

triangle = spark.createDataFrame(data,schema)
triangle.show()

+---+---+---+
|  x|  y|  z|
+---+---+---+
| 13| 15| 30|
| 10| 20| 15|
+---+---+---+



In [0]:
# Report for every three line segments whether they can form a triangle.
# Return the result table in any order.


In [0]:
triangle.alias("t").withColumn(\
    "isTriangle",when(  ( col("t.x")+col("t.y")<=col("t.z") ) | (col("t.y")+col("t.z")<=col("t.x")) | (col("t.z")+col("t.x")<=col("t.y")) ,False)\
                .otherwise(True)   )\
            .show()

+---+---+---+----------+
|  x|  y|  z|isTriangle|
+---+---+---+----------+
| 13| 15| 30|     false|
| 10| 20| 15|      true|
+---+---+---+----------+



In [0]:
# Using udf
@udf(returnType=BooleanType())
def triangleCheck(a,b,c):
    if a+b<=c or b+c<=a or c+a<=b:
        return False
    return True

triangle.alias("t").withColumn("isTriangle",triangleCheck("t.x","t.y","t.z")).show()

+---+---+---+----------+
|  x|  y|  z|isTriangle|
+---+---+---+----------+
| 13| 15| 30|     false|
| 10| 20| 15|      true|
+---+---+---+----------+



In [0]:
triangle.createOrReplaceTempView("tri")
spark.sql("select x,y,z,case when ( (x+y<=z) or (y+z<=x) or (z+x<=y) ) then false else true end as isTriangle from tri").show()

+---+---+---+----------+
|  x|  y|  z|isTriangle|
+---+---+---+----------+
| 13| 15| 30|     false|
| 10| 20| 15|      true|
+---+---+---+----------+



In [0]:
spark.stop()